# 1. Import Libraries

In [29]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import numpy as np
import cv2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.optimizers import Adam

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/leapgestrecog/leapGestRecog/07/02_l/frame_07_02_0067.png
/kaggle/input/leapgestrecog/leapGestRecog/07/02_l/frame_07_02_0153.png
/kaggle/input/leapgestrecog/leapGestRecog/07/02_l/frame_07_02_0105.png
/kaggle/input/leapgestrecog/leapGestRecog/07/02_l/frame_07_02_0018.png
/kaggle/input/leapgestrecog/leapGestRecog/07/02_l/frame_07_02_0050.png
/kaggle/input/leapgestrecog/leapGestRecog/07/02_l/frame_07_02_0029.png
/kaggle/input/leapgestrecog/leapGestRecog/07/02_l/frame_07_02_0195.png
/kaggle/input/leapgestrecog/leapGestRecog/07/02_l/frame_07_02_0051.png
/kaggle/input/leapgestrecog/leapGestRecog/07/02_l/frame_07_02_0074.png
/kaggle/input/leapgestrecog/leapGestRecog/07/02_l/frame_07_02_0142.png
/kaggle/input/leapgestrecog/leapGestRecog/07/02_l/frame_07_02_0149.png
/kaggle/input/leapgestrecog/leapGestRecog/07/02_l/frame_07_02_0121.png
/kaggle/input/leapgestrecog/leapGestRecog/07/02_l/frame_07_02_0194.png
/kaggle/input/leapgestrecog/leapGestRecog/07/02_l/frame_07_02_0003.png
/kaggl

# 2. Define Paths and Load Data
* **Set the Base Directory:**

In [32]:


def load_images_from_folder(base_folder):
    images = []
    labels = []
    
    # Loop over each subject (00, 01, ..., 09)
    for subject in os.listdir(base_folder):
        subject_folder = os.path.join(base_folder, subject)
        
        if os.path.isdir(subject_folder):
            # Loop over each gesture (01_palm, 02_l, etc.)
            for gesture in os.listdir(subject_folder):
                gesture_folder = os.path.join(subject_folder, gesture)
                
                if os.path.isdir(gesture_folder):
                    # Load each image in the gesture folder
                    for img_file in os.listdir(gesture_folder):
                        img_path = os.path.join(gesture_folder, img_file)
                        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)  # Load grayscale images
                        img = cv2.resize(img, (128, 128))  # Resize to 128x128
                        
                        images.append(img)
                        labels.append(gesture)  # Use gesture as label
    
    return np.array(images), np.array(labels)

# Load dataset
base_folder = '/kaggle/input/leapgestrecog/leapGestRecog'  # Replace with the correct path
X, y = load_images_from_folder(base_folder)

* **Normalize and One-Hot Encode Labels:**

In [37]:
# Normalize images
X = X / 255.0  # Scale pixel values to [0, 1]

# Reshape images for CNN input (samples, height, width, channels)
X = X.reshape(X.shape[0], 128, 128, 1)  # 1 for grayscale images

# Encode labels
# Encode labels (this will now encode 10 unique gesture classes)
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)  # Convert string labels to integers
y_categorical = to_categorical(y_encoded, num_classes=10)  # One-hot encode the labels for 10 classes
 # One-hot encode the labels

# 4. Split Data

In [38]:
# Split the data (e.g., 70% train, 15% validation, 15% test)
X_train, X_test, y_train, y_test = train_test_split(X, y_categorical, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=42)


# 5. Build and Compile CNN Model

In [42]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

# Define the CNN model
def build_model():
    model = Sequential()

    # First Conv layer
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 1)))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    # Second Conv layer
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    # Third Conv layer
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    # Flatten the output and pass through fully connected layers
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))

    # Output layer (softmax for multi-class classification)
    model.add(Dense(10, activation='softmax'))  # Change to 10 classes

    return model


# Build and compile the model
model = build_model()
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])


# Print model summary
model.summary()


Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_15 (Conv2D)              │ (None, 126, 126, 32)   │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_15 (MaxPooling2D) │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_16 (Conv2D)              │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_16 (MaxPooling2D) │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_17 (Conv2D)              │ (None, 28, 28, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_17 (MaxPooling2D) │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_5 (Flatten)             │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 128)            │     3,211,392 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 10)             │         1,290 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,305,354 (12.61 MB)

 Trainable params: 3,305,354 (12.61 MB)

 Non-trainable params: 0 (0.00 B)

# 6. Train Model

In [43]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
)

# Train with augmentation
history = model.fit(datagen.flow(X_train, y_train, batch_size=32), 
                    validation_data=(X_val, y_val), 
                    epochs=50, 
                    verbose=1)


Epoch 1/50


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


450/450 ━━━━━━━━━━━━━━━━━━━━ 28s 53ms/step - accuracy: 0.1191 - loss: 2.2835 - val_accuracy: 0.4744 - val_loss: 1.8482
Epoch 2/50
450/450 ━━━━━━━━━━━━━━━━━━━━ 23s 51ms/step - accuracy: 0.3035 - loss: 1.9578 - val_accuracy: 0.5263 - val_loss: 1.5407
Epoch 3/50
450/450 ━━━━━━━━━━━━━━━━━━━━ 24s 52ms/step - accuracy: 0.3560 - loss: 1.7984 - val_accuracy: 0.6119 - val_loss: 1.3412
Epoch 4/50
450/450 ━━━━━━━━━━━━━━━━━━━━ 23s 51ms/step - accuracy: 0.4126 - loss: 1.6805 - val_accuracy: 0.6538 - val_loss: 1.2409
Epoch 5/50
450/450 ━━━━━━━━━━━━━━━━━━━━ 24s 52ms/step - accuracy: 0.4237 - loss: 1.6174 - val_accuracy: 0.6894 - val_loss: 1.1414
Epoch 6/50
450/450 ━━━━━━━━━━━━━━━━━━━━ 23s 51ms/step - accuracy: 0.4574 - loss: 1.5376 - val_accuracy: 0.7063 - val_loss: 1.0706
Epoch 7/50
450/450 ━━━━━━━━━━━━━━━━━━━━ 24s 52ms/step - accuracy: 0.4679 - loss: 1.4643 - val_accuracy: 0.7138 - val_loss: 0.9933
Epoch 8/50
450/450 ━━━━━━━━━━━━━━━━━━━━ 23s 51ms/step - accuracy: 0.4864 - loss: 1.4298 - val_accurac

# 5. Evaluate the Model

In [44]:
# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=2)
print(f'Test accuracy: {test_acc * 100:.2f}%')

125/125 - 0s - 3ms/step - accuracy: 0.9822 - loss: 0.0892
Test accuracy: 98.22%


In [45]:
model.save('hand_gesture_model.h5')